In [16]:
# imports and setup
import os
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
plt.style.use('seaborn-v0_8')
os.makedirs('figures', exist_ok=True)
os.makedirs('data', exist_ok=True)

### 1 - Data Acquisition: Import the Dataset from Kaggle
- Dataset (Kaggle): [steam-reviews](https://www.kaggle.com/datasets/andrewmvd/steam-reviews/data)

- Dataset (local): [data/dataset.csv](data/dataset.csv)


In [ ]:
path = kagglehub.dataset_download("andrewmvd/steam-reviews")

In [ ]:
path

'C:\\Users\\Fatema\\.cache\\kagglehub\\datasets\\andrewmvd\\steam-reviews\\versions\\3'

In [ ]:
! copy $path\dataset.csv data\

# on unix terminal use
# ! cp $path\dataset.csv data\

        1 file(s) copied.


In [ ]:
df = pd.read_csv('data/dataset.csv')

In [6]:
df.head()

,app_id,app_name,review_text,review_score,review_votes
0,10,Counter-Strike,Ruined my life.,1,0
1,10,Counter-Strike,This will be more of a ''my experience with th...,1,1
2,10,Counter-Strike,This game saved my virginity.,1,0
3,10,Counter-Strike,• Do you like original games? • Do you like ga...,1,0
4,10,Counter-Strike,"Easy to learn, hard to master.",1,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6417106 entries, 0 to 6417105
Data columns (total 5 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   app_id        int64 
 1   app_name      object
 2   review_text   object
 3   review_score  int64 
 4   review_votes  int64 
dtypes: int64(3), object(2)
memory usage: 244.8+ MB


In [8]:
df.describe()

,app_id,review_score,review_votes
count,6.417106e+06,6.417106e+06,6.417106e+06
mean,2.274695e+05,6.394992e-01,1.472446e-01
std,1.260451e+05,7.687918e-01,3.543496e-01
min,1.000000e+01,-1.000000e+00,0.000000e+00
25%,2.018100e+05,1.000000e+00,0.000000e+00
50%,2.391600e+05,1.000000e+00,0.000000e+00
75%,3.056200e+05,1.000000e+00,0.000000e+00
max,5.653400e+05,1.000000e+00,1.000000e+00


### 2- EDA

In [18]:
print('Number of games reviewed: {}'.format(df['app_id'].nunique())) # how many games are there?

Number of games reviewed: 9972


In [22]:
# Explore Sentiment (review_score) class balance

### 3- Data Preprocessing

### 4- Model Training

### 5- Results Evaluations